## Global Variables

In [19]:
main_testing_path='E:/Users/1167486/Local/data/data_for_testing/'

## Test Zone
### I've created this notebook to test the functions created for the app

### Emotion functions and alternatives

In [20]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
from pysentimiento import create_analyzer
from transformers import pipeline
import torch # Puede ser necesario instalar pytorch


In [21]:
#### Gemma Generated 
### Nueva función generada de Gemma
import os 

from google import genai
from dotenv import load_dotenv
load_dotenv()


True

In [22]:
GEMINI_API_KEY  = os.getenv('GEMINI_API_KEY')
GOOGLE_API_KEY  = os.getenv('GEMINI_API_KEY')


In [24]:
clientGenai  = genai.Client(api_key= GOOGLE_API_KEY)


Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


In [25]:
clientGenai.models.generate_content(model = 'gemini-2.0-flash', contents='¿Cuál es la capital de México?')

GenerateContentResponse(
  automatic_function_calling_history=[],
  candidates=[
    Candidate(
      avg_logprobs=-0.25623297691345215,
      content=Content(
        parts=[
          Part(
            text="""La capital de México es Ciudad de México.
"""
          ),
        ],
        role='model'
      ),
      finish_reason=<FinishReason.STOP: 'STOP'>
    ),
  ],
  model_version='gemini-2.0-flash',
  sdk_http_response=HttpResponse(
    headers=<dict len=11>
  ),
  usage_metadata=GenerateContentResponseUsageMetadata(
    candidates_token_count=10,
    candidates_tokens_details=[
      ModalityTokenCount(
        modality=<MediaModality.TEXT: 'TEXT'>,
        token_count=10
      ),
    ],
    prompt_token_count=8,
    prompt_tokens_details=[
      ModalityTokenCount(
        modality=<MediaModality.TEXT: 'TEXT'>,
        token_count=8
      ),
    ],
    total_token_count=18
  )
)

In [3]:
spacy_model = 'es_core_news_md'
nlp = spacy.load(spacy_model)
if not nlp.has_pipe('spacytextblob'):
    nlp.add_pipe('spacytextblob')



emotions_analyzer = create_analyzer(task='emotion', lang='es')
sentiments_analyzer = create_analyzer(task='sentiment', lang='es')


sentiment_model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
#emotion_model_name = "Toshifumi/bert-base-multilingual-cased-finetuned-emotion"
emotion_model_name = "rmtariq/multilingual-emotion-classifier"
device = 0 if torch.cuda.is_available() else -1 # Usar GPU si está disponible

emotion_pipeline = pipeline("text-classification", model=emotion_model_name, 
                            device=device )
sentiment_pipeline = pipeline("sentiment-analysis", model=sentiment_model_name, 
                              device=device)


In [4]:
def generate_emotions_analysis(text):
    text = str(text)
    analyzer = emotions_analyzer
    if analyzer is None:
        return "Error interno modelo de emociones"
    emotion_map_es = {
        "joy": "Alegría", 
        "sadness": "Tristeza",
        "anger": "Enojo", 
        "fear": "Miedo", 
        "surprise": "Sorpresa", 
        "disgust": "Asco", 
        "neutral": "Neutral"
    }
    try:
        result = analyzer.predict(text)
        primary_emotion_en = result.output
        if primary_emotion_en=="others":
            primary_emotion_en="neutral"
        return emotion_map_es.get(primary_emotion_en, "Desconocida")
    except Exception as e:
        print('Error en el modelo de emociones {e}')
        return 'Análisis de emociones fallido'

def sentiment_based_on_emotions_analysis(text):
    text = str(text)
    emotions_available = ['Alegría', 'Tristeza', 'Enojo', 'Miedo', 'Sorpresa', 'Asco', 'Neutral', 'Desconocida']
    positive_emotions = ['Alegría', 'Sorpresa']
    negative_emotions = ['Tristeza', 'Enojo', 'Miedo', 'Asco']
    if text in emotions_available:
        if text in positive_emotions:
            return 'Positivo'
        elif text in negative_emotions:
            return 'Negativo'
        elif text=='Neutral':
            return 'Neutral'
    else:
        return 'Neutral'



def generate_sentiment_analysis(text):
    text= str(text) ## Hay que asegurarse que sea texto 
    #### Combinación entre spacy y pysentimiento  por si las dudas dudosas
    nlp_model =nlp 
    if nlp_model:
        doc = nlp_model(text)
        polarity_spacy = doc._.blob.polarity
        if polarity_spacy > 0.1:
            return 'Positivo'  
        elif polarity_spacy < -0.1:
            return 'Negativo' 
        else:
            sentiment_spacy_neutral = 'Neutral'                
            analyzer_pysent = sentiments_analyzer
            if analyzer_pysent: 
                try: 
                    result_pysentimiento = analyzer_pysent.predict(text)
                    pysent_sentiment_map = {'POS': 'Positivo', 'NEG': 'Negativo', 'NEU': 'Neutral'}
                    sentiment_pysent = pysent_sentiment_map.get(result_pysentimiento.output, 'Neutral')
                    if sentiment_pysent != 'Neutral':
                        return sentiment_pysent
                    else:
                        return sentiment_spacy_neutral
                except Exception as e: 
                 return sentiment_spacy_neutral
            else: 
                return sentiment_spacy_neutral
    else: 
        print('NLP no encontrado, ahora usemos pysentimiento')
        analyzer_pysent = sentiments_analyzer

        if analyzer_pysent:
            try:
                result = analyzer_pysent.predict(text)
                pysent_sentiment_map = {"POS": "Positivo", "NEG": "Negativo", "NEU": "Neutral"}
                return pysent_sentiment_map.get(result.output, "Neutral")
            except Exception as e:
                    print(f"Error en pysentimiento: {e}")
                    return "Error: Análisis de sentimiento (Pysentimiento) fallido"
        else: # Both Spacy and Pysentimiento are unavailable
            return "Error: Modelos de sentimiento no disponibles"        



In [5]:
def generate_sentiment_hf(text):
    """
    Función alternativa para generar sentimiento usando un pipeline de Transformers.
    """
    text = str(text)
    
    try:
        results = sentiment_pipeline(text)
        real_score = results[0]['score']
        if real_score>=0.75:
            # El modelo 'nlptown' devuelve estrellas (1 a 5)
            # Podemos mapear esto a Positivo, Negativo, Neutral
            score = int(results[0]['label'].split()[0]) 
            if score > 3:
                return "Positivo"
            elif score < 3:
                return "Negativo"
            else: 
                return 'Neutral'
        else:
            return 'Neutral'
    except Exception as e:
        print(f"Error en el análisis con Transformers: {e}")
        return "Análisis de sentimiento fallido"




def generate_emotion_hf(text):
    """
    Función alternativa para generar sentimiento usando un pipeline de Transformers.
    """
    text = str(text)
    emotion_map_es = {
        "anger": "Enojo", 
        "fear": "Miedo",
        "happy": "Felicidad", 
        "love": "Amor", 
        "sadness": "Tristeza", 
        "surprise": "Sorpresa", 
        "neutral": "Neutral"
    }

    try:
        results = emotion_pipeline(text)
        score = results[0]['score']
        if score >= 0.8:
            return emotion_map_es.get(results[0]['label'], 'Desconocida')
        else:
            return 'Neutral'
    except Exception as e:
        print(f"Error en el análisis con Transformers: {e}")
        return "Análisis de sentimiento fallido"


In [ ]:
model ='gemini-2.0-flash'

In [17]:
#genai.Client(api_key=GOOGLE_API_KEY).models.generate_content(model=model, con)

In [18]:
#clientGenai.models.generate_content(model = model, contents='¿Cuál es la capital de Nueva Zelanda?')

In [84]:
def get_gemini_gemma_evaluation_summary(df_analyzed, genai_model, text_column,  cols_to_compare, model, category = 'sentimiento'):
    '''
        Función para comparar resultados de las columnas y detectar el mejor modelo de cada una 
    '''
    if genai_model is None:
        return "Error: Necesitas cargar y configurar la API de tu LLM "
    summary_parts = []
    cols_to_compare.append(text_column)

    for col in cols_to_compare:
        if col in df_analyzed.columns:
            counts = df_analyzed[col].value_counts().to_string()
            summary_parts.append(f"Resumen de la columna '{col}':\n{counts}\n")
    
    full_summary = "\n---\n".join(summary_parts)
    prompt = f"""
    Eres un analista de datos experto en procesamiento de lenguaje natural.
    He analizado un conjunto de comentarios usando tres métodos diferentes y he obtenido los siguientes resultados agregados:

    {full_summary}

    Basado en esta distribución de resultados, por favor, proporciona una evaluación comparativa:
    1. ¿Cuál es el mejor modelo en comparar {category} 
    1. ¿Qué método de análisis de sentimiento/emocion  parece ser más "estricto" o conservador al asignar sentimientos negativos o positivos?
    2. ¿Hay alguna diferencia notable en la distribución que sugiera que un modelo podría estar capturando más matices que el otro?
    3. Considerando la distribución de emociones, ¿cuál es el tono general de los comentarios?
    4. ¿Qué método recomendarías para un análisis rápido y cuál para uno más detallado, basándote únicamente en estas distribuciones?

    Sé conciso y directo en tu análisis.
    """

    print("Enviando resumen a Gemma para evaluación experta...")
    try:
        response = genai_model.models.generate_content(model = model, contents= prompt)
        return response.text
    except Exception as e:
        return f"Error al contactar a Gemma para la evaluación: {e}"


In [76]:
import pandas as pd 
data_test = pd.read_csv(main_testing_path+ 'sentiment_analysis_dataset.csv')

In [77]:
test = data_test['text'][1]

In [78]:
test

'me siento abrumado'

In [79]:
sentiment_pipeline(test)

[{'label': '2 stars', 'score': 0.41682007908821106}]

In [80]:
### Primero las emoc
print('emotion_original')
import time 



emotion_original


In [ ]:
data_test = data_test.iloc[:10]

In [81]:
print('Comenzando Original Emotion')
time1 = time.time()
data_test['original_emotion'] = data_test['text'].apply(generate_emotions_analysis)
time2 = time.time()
print(f'Duró {(time2-time1)/60} minutos ejecutarse' ) 

print('Comenzando Alternative Emotion')
time1 = time.time()
data_test['alternative_emotion'] = data_test['text'].apply(generate_emotion_hf)
time2 = time.time()
print(f'Duró {(time2-time1)/60} minutos ejecutarse' ) 


Comenzando Original Emotion
Duró 3.524675985177358 minutos ejecutarse
Comenzando Alternative Emotion
Error en el análisis con Transformers: The expanded size of the tensor (1005) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 1005].  Tensor sizes: [1, 514]
Duró 3.3157790581385296 minutos ejecutarse


In [82]:
#### Después los sent xd
print('Comenzando Original Sentiment')
time1 = time.time()
data_test['original_sentiment'] = data_test['text'].apply(generate_sentiment_analysis)
time2 = time.time()
print(f'Duró {(time2-time1)/60} minutos ejecutarse' ) 

print('Comenzando Sentiment based emotion')
time1 = time.time()
data_test['sentiment_based_on_emotion'] = data_test['original_emotion'].apply(sentiment_based_on_emotions_analysis)
time2 = time.time()
print(f'Duró {(time2-time1)/60} minutos ejecutarse' ) 

print('Comenzando Alternative Sentiment')
time1 = time.time()
data_test['alternative_sentiment'] = data_test['text'].apply(generate_sentiment_hf)
time2 = time.time()
print(f'Duró {(time2-time1)/60} minutos ejecutarse' ) 




Comenzando Original Sentiment
Duró 3.9949589172999063 minutos ejecutarse
Comenzando Sentiment based emotion
Duró 0.00016669034957885742 minutos ejecutarse
Comenzando Alternative Sentiment
Error en el análisis con Transformers: The size of tensor a (942) must match the size of tensor b (512) at non-singleton dimension 1
Duró 3.9709062735239664 minutos ejecutarse


In [85]:
data_test

,user,text,date,emotion,sentiment,original_emotion,alternative_emotion,original_sentiment,sentiment_based_on_emotion,alternative_sentiment
0,@erreborda,termine bien abrumado después de hoy,"Jan 6, 2024 · 2:53 AM UTC",overwhelmed,scared,Tristeza,Neutral,Negativo,Negativo,Neutral
1,@shpiderduck,me siento abrumado,"Jan 6, 2024 · 2:35 AM UTC",overwhelmed,scared,Tristeza,Tristeza,Negativo,Negativo,Neutral
2,@Alex_R_art,Me siento un poco abrumado por la cantidad de ...,"Jan 6, 2024 · 12:20 AM UTC",overwhelmed,scared,Miedo,Tristeza,Negativo,Negativo,Neutral
3,@anggelinaa97,Salvador la única persona que no la ha abrumad...,"Jan 5, 2024 · 10:38 PM UTC",overwhelmed,scared,Neutral,Felicidad,Negativo,Neutral,Neutral
4,@diegoreyesvqz,Denme un helado o algo que ando full abrumado.,"Jan 5, 2024 · 8:38 PM UTC",overwhelmed,scared,Neutral,Neutral,Positivo,Neutral,Neutral
...,...,...,...,...,...,...,...,...,...,...
2585,@lavivianaleyva,No podemos vivir con miedo: ¡Manejen borrach...,"Jan 6, 2024 · 3:08 AM UTC",daring,joyful,Enojo,Felicidad,Neutral,Negativo,Neutral
2586,@Carmeen_Alicia,"La vida es un constante, SIN MIEDO AL ÉXITO 💅🏼","Jan 6, 2024 · 3:07 AM UTC",daring,joyful,Neutral,Felicidad,Positivo,Neutral,Neutral
2587,@homicidios_,Esquizofrenia = mente dividida: Miedo a las re...,"Jan 6, 2024 · 2:59 AM UTC",daring,joyful,Neutral,Neutral,Negativo,Neutral,Neutral
2588,@brigethcoba,"""Lo que más miedo me da, es ver cómo desaparec...","Jan 6, 2024 · 2:55 AM UTC",daring,joyful,Miedo,Miedo,Negativo,Negativo,Neutral


In [88]:
get_gemini_gemma_evaluation_summary(data_test.iloc[:2], 
                                    genai_model= clientGenai, 
                                    text_column='text',  cols_to_compare = ['original_emotion', 'alternative_emotion'],
                                    model='gemini-2.0-flash-lite', 
                                    category = 'emocion')

Enviando resumen a Gemma para evaluación experta...


"Error al contactar a Gemma para la evaluación: 403 PERMISSION_DENIED. {'error': {'code': 403, 'message': 'Generative Language API has not been used in project 734555245231 before or it is disabled. Enable it by visiting https://console.developers.google.com/apis/api/generativelanguage.googleapis.com/overview?project=734555245231 then retry. If you enabled this API recently, wait a few minutes for the action to propagate to our systems and retry.', 'status': 'PERMISSION_DENIED', 'details': [{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'SERVICE_DISABLED', 'domain': 'googleapis.com', 'metadata': {'activationUrl': 'https://console.developers.google.com/apis/api/generativelanguage.googleapis.com/overview?project=734555245231', 'containerInfo': '734555245231', 'consumer': 'projects/734555245231', 'service': 'generativelanguage.googleapis.com', 'serviceTitle': 'Generative Language API'}}, {'@type': 'type.googleapis.com/google.rpc.LocalizedMessage', 'locale': 'en-US', 'mess